In [1]:
import numpy as np

In [11]:
d = np.loadtxt("/Users/fxb/Desktop/拼车预测实验/SIMU_RESULT_2000.csv", delimiter=',')

In [12]:
alpha = (d[:, 1:] > 0).astype(int)

In [13]:
deg = np.sum(alpha, axis=1)

In [14]:
init_d = d[:, 1:]
w = np.zeros((init_d.shape[0], init_d.shape[1]))
for j in range(init_d.shape[0]):
    for l in range(init_d.shape[1]):
        w[j, l] = np.inner(alpha[:, j], alpha[:, l] / deg) / deg[l]

In [15]:
new_d = w ** 5 @ init_d

In [21]:
OD_cluster = -np.ones(init_d.shape[0])
cluster_num = 0
for i in range(init_d.shape[0]):
    if OD_cluster[i] == -1:
        OD_cluster[new_d[i, :] > 1e-6] = cluster_num
        cluster_num += 1

In [19]:
np.savetxt("/Users/fxb/Desktop/拼车预测实验/CLUSTER_2000.csv", OD_cluster.reshape((OD_cluster.shape[0], 1)), fmt="%d")

In [22]:
import pickle

In [23]:
with open("/Users/fxb/Desktop/拼车预测实验/WEIGHT_BGRAPH", "wb") as outfile:
    pickle.dump(w, outfile)
with open("/Users/fxb/Desktop/拼车预测实验/INIT_D", "wb") as outfile:
    pickle.dump(init_d, outfile)

In [24]:
with open("/Users/fxb/Desktop/拼车预测实验/WEIGHT_BGRAPH", "rb") as outfile:
    in_w = pickle.load(outfile)

In [25]:
new_d = new_d + new_d.T

K = 20
NODE_NUM = new_d.shape[0]
MAX_ITER = 100
node_cluster = np.random.randint(low=0, high=K, size=NODE_NUM)
medoids = np.random.choice(np.arange(0, NODE_NUM), K, False)
iter_time = 0
while iter_time < MAX_ITER:
    is_converge = True
    for i in range(NODE_NUM):
        single_distance = new_d[i, medoids]
        if node_cluster[i] != np.argmax(single_distance):
            node_cluster[i] = np.argmax(single_distance)
            is_converge = False
    # avg_radius = np.zeros(K)
    for k in range(K):
        distance_of_kth_cluster = new_d[node_cluster == k, :]
        distance_of_kth_cluster = distance_of_kth_cluster[:, node_cluster == k]
        # avg_radius[k] = np.mean(distance_of_kth_cluster)
        min_distance_node_idx = np.argmax(np.sum(distance_of_kth_cluster, axis=0))
        min_distance_node_idx = np.arange(NODE_NUM)[node_cluster == k][min_distance_node_idx]
        if medoids[k] != min_distance_node_idx:
            medoids[k] = min_distance_node_idx
            is_converge = False
    # print(avg_radius)
    if is_converge:
        break
    iter_time += 1

In [27]:
np.savetxt("/Users/fxb/Desktop/拼车预测实验/CLUSTER_2000.csv", node_cluster.reshape((node_cluster.shape[0], 1)), fmt="%d")

In [29]:
import csv
OD_NUM = 2000
OD_filepath = "/Users/fxb/Desktop/拼车预测实验/OD_2000.csv"
OD_pairs = np.zeros((OD_NUM, 6))
with open(OD_filepath, mode='r') as file:
    data = csv.reader(file)
    head = next(data)
    id_col = head.index("id")
    x1_col = head.index("originX")
    y1_col = head.index("originY")
    x2_col = head.index("destX")
    y2_col = head.index("destY")
    lam_col = head.index("lambda")
    j = 0
    for row in data:
        x1 = int(row[x1_col])
        x2 = int(row[x2_col])
        y1 = int(row[y1_col])
        y2 = int(row[y2_col])
        lam = float(row[lam_col])
        OD_pairs[j, :] = np.array([[j, x1, y1, x2, y2, lam]])
        j += 1
        if j >= OD_NUM:
            break

In [47]:
MAX_OD_NUM = 20
for n in range(MAX_OD_NUM):
    new_OD_pairs = OD_pairs[init_d[n, :] > 0, :]
    np.savetxt("/Users/fxb/Desktop/误差测试2/OD_Cluster_" + str(n) + ".csv", new_OD_pairs, fmt=('%d', '%d', '%d', '%d', '%d', '%.2f'), delimiter=',', header="id, originX, originY, destX, destY, lambda", comments='')

In [48]:
index = -np.ones(MAX_OD_NUM)
for n in range(MAX_OD_NUM):
    index[n] = np.sum(init_d[n, :n] > 0)

In [49]:
predict_result = np.zeros((MAX_OD_NUM, 3))
for n in range(MAX_OD_NUM):
    with open("/Users/fxb/Desktop/误差测试2/OD_Cluster_Result_" + str(n) + ".csv", mode='r') as file:
        data = csv.reader(file)
        head = next(data)
        pw_col = head.index("Pw")
        lw_col = head.index("Lw")
        ew_col = head.index("ew")
        j = 0
        for row in data:
            if j < index[n]:
                j += 1
                continue
            pw = float(row[pw_col])
            lw = float(row[lw_col])
            ew = float(row[ew_col])
            predict_result[n, 0] = pw
            predict_result[n, 1] = lw
            predict_result[n, 2] = ew
            break

In [50]:
OD_filepath = "/Users/fxb/Desktop/拼车预测实验/RESULT_2000.csv"
global_predict_result = np.zeros((MAX_OD_NUM, 3))
with open(OD_filepath, mode='r') as file:
    data = csv.reader(file)
    head = next(data)
    pw_col = head.index("Pw")
    lw_col = head.index("Lw")
    ew_col = head.index("ew")
    j = 0
    for row in data:
        pw = float(row[pw_col])
        lw = float(row[lw_col])
        ew = float(row[ew_col])
        global_predict_result[j, 0] = pw
        global_predict_result[j, 1] = lw
        global_predict_result[j, 2] = ew
        j += 1
        if j >= MAX_OD_NUM:
            break

In [51]:
rel_error_2 = np.abs(global_predict_result - predict_result) / global_predict_result

In [46]:
np.arange(2000)[init_d[0, :] > 0]

array([   0,   24,  158,  372,  477,  571,  678,  848,  942, 1000, 1022,
       1024, 1034, 1372, 1477, 1643, 1892])

In [52]:
rel_error_2 - rel_error

array([[ 5.53974601e-03,  1.86064982e-03,  3.39305054e-02],
       [ 4.35054379e-02,  2.51662062e-03,  6.18079315e-02],
       [ 9.22900923e-03, -1.39853372e-03,  2.09487635e-02],
       [ 3.26357693e-02,  6.98218749e-03,  6.08858019e-02],
       [ 6.80897644e-03, -2.23020849e-04,  7.31015531e-02],
       [ 8.00226258e-02,  1.96489541e-02,  8.43303868e-02],
       [ 1.10251814e-02,  3.14136126e-04,  8.53829587e-03],
       [ 4.01514455e-03,  8.62623131e-05,  2.22385649e-03],
       [ 1.27500638e-03,  3.32474642e-03,  2.65207707e-02],
       [ 4.79236964e-03,  1.15910193e-02,  1.29064070e-02],
       [ 4.67989982e-02,  2.28425370e-02,  2.61989395e-02],
       [ 7.00122404e-03,  7.12312099e-03,  3.12914530e-03],
       [ 8.53851032e-03,  6.41830215e-05, -1.72528102e-03],
       [ 1.59544758e-02,  5.11068548e-03,  2.40240083e-02],
       [ 9.27858269e-04,  3.12000752e-03,  6.86790456e-03],
       [ 1.12163796e-02,  1.81420642e-03,  1.64075678e-03],
       [ 1.56797902e-04, -5.51675796e-04